In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://www.uvic.ca/calendar/future/undergrad/index.php#/programs/SkYVTmCzE?bc=true&bcCurrent=Mechanical%20Engineering&bcItemType=programs"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

# 1. Collect course link texts for driver to click on

In [4]:
driver.get(url)
time.sleep(5)
page_soup = soup(driver.page_source, 'lxml')

In [5]:
core_container = page_soup.findAll("div", {"class": "noBreak"})[3]

In [6]:
links = core_container.findAll("a")
link_texts = [link.text for link in links]
link_texts

['CHEM150',
 'CSC111',
 'ENGR110',
 'ENGR120',
 'ENGR130',
 'ENGR141',
 'MATH100',
 'MATH109',
 'MATH101',
 'MATH110',
 'PHYS110',
 'PHYS111',
 'CSC116',
 'ECE216',
 'ECE250',
 'ENGR297',
 'MATH200',
 'MATH204',
 'MECH200',
 'MECH220',
 'MECH240',
 'MECH242',
 'MECH285',
 'STAT254',
 'CSC349A',
 'ECE365',
 'ECON180',
 'MECH320',
 'MECH330',
 'MECH335',
 'MECH345',
 'MECH350',
 'MECH360',
 'MECH380',
 'MECH390',
 'MECH395',
 'ENGR446',
 'ENGR498',
 'MECH400',
 'MECH462',
 'MECH410',
 'MECH442',
 'MECH455',
 'MECH458',
 'MECH460',
 'MECH475',
 'MECH493']

In [7]:
len(link_texts)

47

In [8]:
electives_container = page_soup.findAll("div", {"class": "noBreak"})[5]

In [9]:
link_texts += [container.text for container in electives_container.findAll("a")]
link_texts

['CHEM150',
 'CSC111',
 'ENGR110',
 'ENGR120',
 'ENGR130',
 'ENGR141',
 'MATH100',
 'MATH109',
 'MATH101',
 'MATH110',
 'PHYS110',
 'PHYS111',
 'CSC116',
 'ECE216',
 'ECE250',
 'ENGR297',
 'MATH200',
 'MATH204',
 'MECH200',
 'MECH220',
 'MECH240',
 'MECH242',
 'MECH285',
 'STAT254',
 'CSC349A',
 'ECE365',
 'ECON180',
 'MECH320',
 'MECH330',
 'MECH335',
 'MECH345',
 'MECH350',
 'MECH360',
 'MECH380',
 'MECH390',
 'MECH395',
 'ENGR446',
 'ENGR498',
 'MECH400',
 'MECH462',
 'MECH410',
 'MECH442',
 'MECH455',
 'MECH458',
 'MECH460',
 'MECH475',
 'MECH493',
 'MECH410',
 'MECH420',
 'MECH460',
 'MECH466',
 'MECH495',
 'MECH497',
 'MECH472',
 'MECH473',
 'MECH481',
 'MECH448',
 'MECH452',
 'MECH472',
 'MECH481',
 'MECH483',
 'MECH443',
 'MECH444',
 'MECH445',
 'MECH446',
 'MECH447',
 'MECH449',
 'MECH459',
 'MECH493',
 'MECH494',
 'MECH497',
 'MECH443',
 'MECH444',
 'MECH446',
 'MECH447',
 'MECH475',
 'MECH492',
 'MECH493',
 'MECH494',
 'MECH495',
 'MECH421',
 'MECH430',
 'MECH455',
 'MECH458

In [10]:
link_texts = list(dict.fromkeys(link_texts))
len(link_texts)

77

# 2. Test run - try to scrape first course

In [11]:
driver.get(url)
time.sleep(5)
driver.find_element_by_link_text(link_texts[0]).click()
time.sleep(5)
driver.current_url

'https://www.uvic.ca/calendar/future/undergrad/index.php#/courses/view/5d1f6e3494e82e2400a23437'

In [12]:
page_soup = soup(driver.page_source, 'lxml')

In [13]:
container = page_soup.find("div", {"class": "course-view__itemDetailContainer___2tFFK"})
course_desc = container.find("p").text.strip()
course_desc

'Atomic and molecular structure; periodicity, chemical bonding; gases, liquids, and solids; phase equilibria; equilibrium; thermochemistry; chemical thermodynamics; electrochemistry.'

In [14]:
course_title = container.find("h2").text.strip()
course_title

'CHEM150 - Engineering Chemistry'

In [15]:
course_code = course_title.split(" - ")[0]
course_code

'CHEM150'

In [16]:
course_name = course_title.split(" - ")[1]
course_name

'Engineering Chemistry'

# 3. Test run successful. Implement automation script to scrape all courses

In [19]:
from selenium.common.exceptions import NoSuchElementException

course_codes = []
course_names = []
course_descs = []
counter = 0

driver.get("https://www.uvic.ca/calendar/future/undergrad/index.php#/programs/SkYVTmCzE?bc=true&bcCurrent=Mechanical%20Engineering&bcItemType=programs")
time.sleep(5)

for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("encountered no link for {}".format(link_text), ", retrying")
        driver.refresh()
        time.sleep(5)
        
        try:
            link = driver.find_element_by_link_text(link_text)
        except:
            NoSuchElementException
            print("no link for {}".format(link_text))
            continue

    time.sleep(2)
    link.click()    
    time.sleep(3)
    
    #scrape course info
    try:
        page_soup = soup(driver.page_source, 'lxml')
        container = page_soup.find("div", {"class": "course-view__itemDetailContainer___2tFFK"})
        course_title = container.find("h2").text.strip()
    except AttributeError:
        print("encountered Attribute error")
        driver.refresh()
        time.sleep(5)
        
        page_soup = soup(driver.page_source, 'lxml')
        container = page_soup.find("div", {"class": "course-view__itemDetailContainer___2tFFK"})
        course_title = container.find("h2").text.strip()       
    
    course_code = course_title.split(" - ")[0]
    course_name = course_title.split(" - ")[1]
    course_desc = container.find("p").text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1])
    counter += 1
    
    #go to course list page
    driver.back()
    time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  CHEM150
Scraped  CSC111
Scraped  ENGR110
Scraped  ENGR120
Scraped  ENGR130
Scraped  ENGR141
Scraped  MATH100
Scraped  MATH109
Scraped  MATH101
Scraped  MATH110
Scraped  PHYS110
Scraped  PHYS111
Scraped  CSC116
Scraped  ECE216
encountered no link for ECE250 , retrying
Scraped  ECE250
Scraped  ENGR297
Scraped  MATH200
Scraped  MATH204
Scraped  MECH200
Scraped  MECH220
Scraped  MECH240
Scraped  MECH242
Scraped  MECH285
Scraped  STAT254
Scraped  CSC349A
Scraped  ECE365
Scraped  ECON180
Scraped  MECH320
Scraped  MECH330
Scraped  MECH335
Scraped  MECH345
Scraped  MECH350
Scraped  MECH360
Scraped  MECH380
Scraped  MECH390
Scraped  MECH395
Scraped  ENGR446
Scraped  ENGR498
Scraped  MECH400
Scraped  MECH462
Scraped  MECH410
Scraped  MECH442
Scraped  MECH455
Scraped  MECH458
encountered Attribute error
Scraped  MECH460
Scraped  MECH475
Scraped  MECH493
Scraped  MECH420
Scraped  MECH466
Scraped  MECH495
Scraped  MECH497
Scraped  MECH472
Scraped  MECH473
Scraped  MECH481
Scraped  MECH448


# 4. Inspect and write CSV

In [20]:
course_codes

['CHEM150',
 'CSC111',
 'ENGR110',
 'ENGR120',
 'ENGR130',
 'ENGR141',
 'MATH100',
 'MATH109',
 'MATH101',
 'MATH110',
 'PHYS110',
 'PHYS111',
 'CSC116',
 'ECE216',
 'ECE250',
 'ENGR297',
 'MATH200',
 'MATH204',
 'MECH200',
 'MECH220',
 'MECH240',
 'MECH242',
 'MECH285',
 'STAT254',
 'CSC349A',
 'ECE365',
 'ECON180',
 'MECH320',
 'MECH330',
 'MECH335',
 'MECH345',
 'MECH350',
 'MECH360',
 'MECH380',
 'MECH390',
 'MECH395',
 'ENGR446',
 'ENGR498',
 'MECH400',
 'MECH462',
 'MECH410',
 'MECH442',
 'MECH455',
 'MECH458',
 'MECH460',
 'MECH475',
 'MECH493',
 'MECH420',
 'MECH466',
 'MECH495',
 'MECH497',
 'MECH472',
 'MECH473',
 'MECH481',
 'MECH448',
 'MECH452',
 'MECH483',
 'MECH443',
 'MECH444',
 'MECH445',
 'MECH446',
 'MECH447',
 'MECH449',
 'MECH459',
 'MECH494',
 'MECH492',
 'MECH421',
 'MECH430',
 'MECH485',
 'MECH450A',
 'MECH450B',
 'MECH450C',
 'MECH450D',
 'MECH450E',
 'MECH450F',
 'MECH498',
 'MECH499']

In [21]:
course_names

['Engineering Chemistry',
 'Fundamentals of Programming with Engineering Applications',
 'Design and Communication I',
 'Design and Communication II',
 'Introduction to Professional Practice',
 'Engineering Mechanics',
 'Calculus I',
 'Introduction to Calculus',
 'Calculus II',
 'Matrix Algebra for Engineers',
 'Introductory Physics I',
 'Introductory Physics II',
 'Fundamentals of Programming with Engineering Applications II',
 'Electricity and Magnetism',
 'Linear Circuits I',
 'Technology and Society',
 'Calculus III',
 'Calculus IV',
 'Engineering Drawing, CAD and Manufacturing',
 'Mechanics of Solids I',
 'Thermodynamics',
 'Dynamics',
 'Properties of Engineering Materials',
 'Probability and Statistics for Engineers',
 'Numerical Analysis',
 'Applied Electronics and Electrical Machines',
 'Introduction to Economics and Financial Project Evaluation',
 'Mechanics of Solids II',
 'Introduction to Mechanical Vibrations',
 'Theory of Mechanisms',
 'Mechanics of Fluids',
 'Engineering 

In [23]:
course_descs

['Atomic and molecular structure; periodicity, chemical bonding; gases, liquids, and solids; phase equilibria; equilibrium; thermochemistry; chemical thermodynamics; electrochemistry.',
 'Fundamentals of computer programming with real-world engineering examples using an imperative programming language. Topics include variables, data types, statements, control structures, functions, parameter passing mechanisms, expressions, data structures, files, pointers, storage allocation, elementary searching and sorting, recursion, encapsulation, modularity, incremental development, testing and debugging.',
 'Introductory principles of engineering design processes through practical projects to be undertaken by teams of students; integrated development and demonstration of writing, research, design and presentation skills through research and design projects. Writing, research and organizational skills appropriate for University level writing.',
 'Principles of engineering science and design appli

In [25]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df

,Course Number,Course Name,Course Description
0,CHEM150,Engineering Chemistry,"Atomic and molecular structure; periodicity, c..."
1,CSC111,Fundamentals of Programming with Engineering A...,Fundamentals of computer programming with real...
2,ENGR110,Design and Communication I,Introductory principles of engineering design ...
3,ENGR120,Design and Communication II,Principles of engineering science and design a...
4,ENGR130,Introduction to Professional Practice,Awareness of the role and responsibilities of ...
...,...,...,...
72,MECH450D,Special Topics,Presents material in an emerging field or one ...
73,MECH450E,Special Topics,Presents material in an emerging field or one ...
74,MECH450F,Special Topics,Presents material in an emerging field or one ...
75,MECH498,Honours Thesis,Provides an opportunity to carry out a major d...


In [26]:
df.to_csv('UVictoria_MechEng_Core_and_Electives_(AllYears)_Courses.csv', index = False)

In [27]:
driver.quit()